# Innlevering 1 - DAT801

1. Her lastes det inn nødvendige biblioteker.

In [ ]:
%matplotlib inline
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

2. Her settes path til lokal maskin og datasettene leses inn som pandas dataframes. 

In [ ]:
NB_DIR = Path.cwd()
DATA = NB_DIR/'data'
DATA.mkdir(exist_ok=True)

In [ ]:
train = pd.read_csv(DATA/'train.csv')
test = pd.read_csv(DATA/'test.csv')
sampleSubmission = pd.read_csv(DATA/'sample_submission.csv')

3. Kikker på histogram for å få et innblikk i hvilke targets som har høyest representasjon, før datasettet formateres i features og targets, samt fjerner id kolonnen fra features.

In [ ]:
train["target"].hist()
plt.show()

In [ ]:
corr = train.corr() #Sjekker korrelasjon mellom features for å eventuelt selektere bort features.
corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
corr_matrix = train.corr() #sjekker korrelasjon mellom features og target.
print(corr_matrix["target"].sort_values(ascending=False))

In [ ]:
X_train = train.drop(["id", "target"], axis = 1)

In [ ]:
y_train = train["target"].values

In [ ]:
X_test = test.drop(["id"], axis = 1)

In [ ]:
X_train.info()

In [ ]:
np.info(y_train)

In [ ]:
X_test.info

4. Her imputeres det manglende verdier i datasettet ved å bruke median verdi. 

In [ ]:
imp = SimpleImputer(strategy = "median")

In [ ]:
X_train_imputed = imp.fit_transform(X_train)

In [ ]:
X_test_imputed = imp.transform(X_test)

5. Her skaleres datasettet med Standardscaler.

In [ ]:
std = StandardScaler()

In [ ]:
X_train_std = std.fit_transform(X_train_imputed)

In [ ]:
X_train_std = pd.DataFrame(data=X_train_std,columns=X_train.columns )

In [ ]:
X_test_std = std.transform(X_test_imputed)

6. Kikker på datasettet etter det er blitt imputert og skalert.

In [ ]:
X_train_std.info()

In [ ]:
X_train_std.head()

In [ ]:
X_train_std.describe()

7. Bruker RandomForestClassifier som modell, og kjører GridSearchCV for å finne beste parametre for modellen. Bruker så disse for å predikere på testdatasettet.

In [ ]:
rf = RandomForestClassifier()

In [ ]:
param_grid = {"n_estimators": [800],
             "max_depth": [18],
             "max_features": [5],
             "min_samples_leaf": [3],
             "min_samples_split": [10],
             "criterion": ["gini", "entropy"]
             }

In [ ]:
gs_reg = GridSearchCV(estimator=rf, param_grid = param_grid, cv=3, verbose=1, n_jobs=-1)

In [ ]:
gs_reg.fit(X_train_std, y_train)

In [ ]:
best_reg = gs_reg.best_estimator_

In [ ]:
best_reg.get_params()

In [ ]:
y_pred_best = best_reg.predict(X_test_std)

In [ ]:
y_pred_best

8. Lager submissionfilen ved å hente ut id fra testfilen supplert, og knytte det deretter sammen med predikasjoner produsert i en dataframe, før den konverteres til en excelfil som skal lastes opp hos Kaggle.

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'target': y_pred_best})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission_lkk_9.csv', index=False)

Final notes. Dersom dette hadde blitt levert som 1 datasett(1 fil, og ikke noe Kaggle konkurranse), ville jeg brukt stratifisert train,test,split på datasettet. Jeg ville da ha rensket i nullverdier/korrelerte features og lignende før jeg hadde splittet opp datasettet.

Jeg ville da også hatt muligheten for å score underveis, samt laget en confusion matrix for å se på treffsikkerheten til modellen min med TP, FP, TN, FN. Med tanke på at jeg nå ikke har en y_test tilgjengelig å sammenligne med på måten jeg har løst oppgaven min, så får jeg ikke gjort dette. Hadde jeg hatt bedre tid, ville jeg også ha testet stacking av modeller sammen med en booster og trent modellen på nytt. 

Jeg har i ulike notebooks testet alt fra Voting ensembles til pycaret. Jeg legger ved disse notebookene i tillegg til denne, bare slik at du kan se hva jeg har holdt på å utforske i andre tilfeller som ikke har vært helt vellykket. Grunnen til at disse ikke har ført frem, er at jeg har vært litt for ambisiøs i mine intensjoner om å stacke og optimalisere, og all tiden det samlet har tatt å trene ulike modeller, for så å kræsje eller aldri bli ferdig har gjort meg en del erfaringer rikere og innehaver av noen ekstra grå hår. :-) 